<a href="https://colab.research.google.com/github/ORivero-Romero/Deep_Learning/blob/main/03-Arquitectura%20de%20linea%20base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Install kaggle library
!pip install kaggle

In [ ]:
# Upload kaggle credential
from google.colab import files

uploaded = files.upload()

In [ ]:
# Prepare kaggle environment
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle datasets download -d kaustubhb999/tomatoleaf

In [ ]:
!unzip tomatoleaf.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Crear un generador para preprocesar las imágenes
datagen = ImageDataGenerator(rescale=1./255)  # Normalizar los valores a [0, 1]

# Ruta de los datos
train_dir = "/content/tomato/train"
val_dir = "/content/tomato/val"

# Crear generadores para entrenamiento y validación
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Redimensionar imágenes a 150x150
    batch_size=32,
    class_mode='categorical'  # Si las clases son múltiples
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
augmented_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'  # Completar espacios vacíos con bordes o reflejos
)

train_generator = augmented_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Nota: No es común aplicar aumentación a datos de validación
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Extraer etiquetas generadas por los generadores
train_labels = train_generator.classes  # Etiquetas de entrenamiento
val_labels = val_generator.classes  # Etiquetas de validación

# Mapear las clases a índices para saber su orden
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}  # De índice a clase

# Codificar etiquetas en one-hot (opcional, ya que flow_from_directory las usa automáticamente)
train_labels_onehot = to_categorical(train_labels)
val_labels_onehot = to_categorical(val_labels)

# Verificar resultados
print(f"Etiquetas de entrenamiento (numéricas): {train_labels[:5]}")
print(f"Etiquetas de entrenamiento (one-hot):\n{train_labels_onehot[:5]}")
print(f"Índice a clase: {index_to_class}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Crear el modelo
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))  # Más neuronas
model.add(Dropout(0.5))  # Aumentar el Dropout
model.add(Dense(10, activation='softmax'))  # Mantener el número de clases

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.80):
          print("\nReached 80.0% val_accuracy so cancelling training!")
          self.model.stop_training = True

In [ ]:
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs = 50,
                    verbose = 1,
                    callbacks=callbacks)

In [ ]:
# Evaluar el modelo en datos de validación
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Pérdida en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")

In [ ]:
# Visualización de resultados de entrenamiento
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.legend()
plt.title('Pérdida')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.show()

plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de validación')
plt.legend()
plt.title('Precisión')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_image(image_path, model):
    # Cargar y preprocesar la imagen
    img = load_img(image_path, target_size=(150, 150))  # Asegúrate de que la imagen se redimensiona correctamente
    img_array = img_to_array(img)  # Convertir la imagen a un array
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión extra para el batch
    img_array /= 255.0  # Normalizar la imagen (esto depende del preprocesamiento que hayas hecho en tu modelo)

    # Hacer la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=-1)  # Obtener la clase predicha

    return predicted_class[0]

def get_class_label(predicted_class, index_to_class):
    return index_to_class[predicted_class]

def visualize_prediction(image_path, predicted_class_label):
    # Cargar la imagen (en su tamaño original) para visualización
    img = load_img(image_path)
    plt.imshow(img)

    # Establecer el título con la etiqueta de la clase predicha
    plt.title(f"Predicted: {predicted_class_label}")
    plt.axis('off')

    # Mostrar la imagen
    plt.show()

In [ ]:
# Ejemplo de uso de las funciones de predicción
image_path = "/content/tomato/val/Tomato___healthy/Tomato___healthy_original_01c1da17-8d9f-4d69-8a1e-58d37453d3c3___RS_HL 9641.JPG_0a809db8-a6c6-466a-938b-93076625ac71.JPG"
predicted_class = predict_image(image_path, model)
predicted_class_label = get_class_label(predicted_class, index_to_class)

# Visualizar la predicción
visualize_prediction(image_path, predicted_class_label)
